# New Analysis (2017-02-16)

이는 여러 text mining과 machine learning technologies를 위한 프로젝트이다. new classification과 analysis 생산에 적용할 수 있다.

------------

- 초기 데이터 소스 : "Trade News Today" 이메일<br>
- 코드와 테크닉 참고 :
    - [Document Clustering with Python](http://brandonrose.org/clustering) (원본 코드는 최대한 원본스럽게 보존하려 했다).<br>
- Classicification:
    - 뉴스 내용을 카테고리로 구분하는 것에 unsupervised learning, clustering, topic model을 사용했다.<br>
    - 분류된 데이터는 classification models (다른 알고리즘)을 위한 트레이닝 데이터로서 사용될 수 있다.<br>
    - classification model은 뉴스 플로우를 카테고리로 구분짓는데 사용할 수 있다.<br>
- Topic model:
    - 키워드 인식에 topic model을 사용했다. 그 다음 키워드를 사용해서 새로운 데이터를 요약한다(?)<br>
- Sentiment analysis

-------

## Content

_**나중에 끝내겠음 (미완 상태인듯)**_


* [Data extraction, cleaning](#Data-extraction,-cleaning)
    * [Convert emails to TXT files](#Convert-emails-to-TXT-files)
    * [Extract articles from a TXT file and convert to DF](#Extract-articles-from-a-TXT-file-and-convert-to-DF)
    
* Document Clustering with Python
    * [Stopwords, stemming, and tokenization](#Stopwords,-stemming,-and-tokenizing)
    * [Tf-idf and document similarity](#Tf-idf-and-document-similarity)
    * [K-means clustering](#K-means-clustering)
    * [Multidimensional scaling](#Multidimensional-scaling)
    * [Visualizing document clusters](#Visualizing-document-clusters)
    * [Hierarchical document clustering](#Hierarchical-document-clustering)
    * [Latent Dirichlet Allocation (LDA)](#Latent-Dirichlet-Allocation)


In [ ]:
import pandas as pd
# import datefinder
# import numpy as np
# import win32com.client
# import codecs
# import os
# import re
#import itertools

## 데이터 추출과 정제

(이 파일 말고 다른 Spyder-python code를 사용해서) 이메일로부터 내용 추출 과정은 끝냈다. 프로세싱 과정은:<br><br>

**1단계. Outlook 이메일을 텍스트 파일로 변환**<br>
- 이메일은 Outlook 폴더 "@ News" >> "2015" 안에 있어야 한다.<br>
- outlook access가 되어 있는 MS 오피스에서만 돌아간다.<br>
- 한 이메일 세트당 코드는 한 번만 돌려야 한다. (2015년의 이메일들)<br>
- 텍스트 파일들은 UTF-8 대신 ASCII coding으로 저장했다.<br><br>

**2단계. TXT 파일로부터 내용을 추출하고 DF로 전환한다.**<br>
- Extractor는 TXT 파일을 읽고 파싱한다.<br>
- 뉴스 타이틀은 index로 이메일의 맨 위에 listed 한다. Extractor는 내용 식별을 위해 내용 몸체 위의 각 타이틀을 index의 headings과 매치한다.<br>
- 몇몇 내용은 매치하는데 문제가 있을 수 있음 : 그럴땐 TXT 파일을 수정하고 비정상 텍스트들을 교정한다. 그리고는 모든게 잘 매칭될때까지 다시 돌려본다. <br>
- Progress: 데이터 완료: 2015 와 2016<br><br>

**3단계. processing 추가**<br>
- 언어 발견<br>
- 날짜 추가<br>
- 새로운 소속 발견

Spyder-Python으로 만들어진 pck로부터 데이터를 읽어온다

In [ ]:
df = pd.read_pickle('df_save.pck')
df.head()

가능한 추가 과정을 아래 데이터 분석 프로세스에서 사용한다. 일부는 Spyder-Python 프로그램으로 이전할 수 있다.

In [ ]:
df = df[df['Language'].isin(['en'])]
# Combine title and contents
df['Text'] = df['Title'] + ' ' + df['Content']

In [ ]:
df[df['Text'].str.contains('S.Korea')].head()

In [ ]:
# Normalize words

replacements = {
    'Text': {
        r'\'s': '', 
        'Indian': 'India', 
        'nextgeneration': 'next generation', 
        '//iconnect\.wto\.org/': '', 
        '-': ' ', 
        'U.S.': 'United States', 
        'US': 'United States', 
        'S.Korea': 'South Korea', 
        'S. Korea': 'South Korea', 
        'WTO': 'world trade organization', 
        '‘': '', 
        'imports': 'import', 
        'Imports': 'import', 
        'exports': 'export', 
        'Exports': 'export', 
        'NZ ': 'New Zealand ', 
        '\"': '',
        '\'': '', 
    }
}

df.replace(replacements, regex=True, inplace=True)


In [ ]:
# Test
df[df['Text'].str.contains('S.Korea')].head()

DF에서 리스트로 변환

In [ ]:
texts = df['Text'].tolist()
titles = df['Title'].tolist()
dates = df['Date'].tolist()
articlecodes = df['ArticleCode'].tolist()

In [ ]:
print(str(len(texts)) + ' texts')
print(str(len(titles)) + ' titles')
print(str(len(dates)) + ' dates')
print(str(len(articlecodes)) + ' articlecodes')

## 분석

### 원본 노트들<br>

이 가이드에서, 파이썬을 사용해서 문서 세트를 어떻게 cluster하는지 설명한다. 상위 100개 필름들의 시놉시스(하나의 IMDB 리스트당)안의 잠재 구조를 식별하는 예시다. 이 예시에 대해서 더 자세한 설명이 필요하다면 [the original post](http://www.brandonrose.org/top100)를 봐라. 이 가이드는:


<ul>
<li> 각 시놉시스별 tokenizing과 stemming
<li> [tf-idf](http://en.wikipedia.org/wiki/Tf%E2%80%93idf)를 이용해서 corpus를 벡터 공간으로 변경<br>
<li> 유사도 측정으로 각 문서별 코사인 거리 계산<br>
<li> [k-means algorithm](http://en.wikipedia.org/wiki/K-means_clustering)을 사용한 문서 클러스터링 <br>
<li> corpus 내부의 차원 축소를 위해 [multidimensional scaling](http://en.wikipedia.org/wiki/Multidimensional_scaling) 사용<br>
<li> [matplotlib](http://matplotlib.org/) 과 [mpld3](http://mpld3.github.io/)를 사용해서 클러스터링 결과 plotting <br>


<li> [Ward clustering](http://en.wikipedia.org/wiki/Ward%27s_method)을 사용해 corpus에 계층적 클러스터링 실시 <br>
<li> plotting a Ward dendrogram<br>
<li> [Latent Dirichlet Allocation (LDA)](http://en.wikipedia.org/wiki/Latent_Dirichlet_allocation)을 사용해 topic modeling
</ul>

### 콘텐츠

<ul>
<li>[Stopwords, stemming, and tokenization](#Stopwords,-stemming,-and-tokenizing)
<li>[Tf-idf and document similarity](#Tf-idf-and-document-similarity)
<li>[K-means clustering](#K-means-clustering)
<li>[Multidimensional scaling](#Multidimensional-scaling)
<li>[Visualizing document clusters](#Visualizing-document-clusters)
<li>[Hierarchical document clustering](#Hierarchical-document-clustering)
<li>[Latent Dirichlet Allocation (LDA)](#Latent-Dirichlet-Allocation)
</ul>

In [ ]:
import numpy as np
import pandas as pd
import nltk
nltk.download('stopwords')
from bs4 import BeautifulSoup
import re
import os
import codecs
from sklearn import feature_extraction
import mpld3

# 불용어(Stopwords), stemming과 tokenizing


**Stopwords(불용어)**

 영어 불용어 리스트 사용을 위해 [NLTK's](http://www.nltk.org/) 로딩. [Stop words](http://en.wikipedia.org/wiki/Stop_words)는 명확한 뜻이 없는 "in"이나 "a","the"같은 단어들이다.

In [ ]:
# load nltk's English stopwords as variable called 'stopwords'
my_stop_words = nltk.corpus.stopwords.words('english')

# Add my stopwords
my_stop_words = my_stop_words + ['world_trade_organization','years','year','said','important',
                                 'new','would','','','','']

# Stopwords of country names  NOT WORKING ????
#my_stop_words = my_stop_words + ['world_trade_organization','years','year','said','important','new','would','united_states',
#                                   'japan','india','obama','canada','mexico','russia','eu','european','china','chinese','would']

**MWETokenizer로 단어들을 이어 붙일 수 있다**

In [ ]:
from nltk.tokenize import MWETokenizer
tokenizer = MWETokenizer([('world', 'bank'), ('world', 'trade', 'organization'), ('doha', 'round'),
                          ('united', 'states'), ('european', 'union'), ('new', 'zealand'),
                          ('per', 'cent'),('south', 'korea'),
                          ])

# Test the tokenizer
tokenizer.tokenize('In a little or a european union little bit world trade organization'.split())


In [ ]:
# Test the function
#tokenize_and_stem('In World Bank or a_little. bit  _ World Trade Organization. United States')

## load nltk's SnowballStemmer as variabled 'stemmer'
#from nltk.stem.snowball import SnowballStemmer
#stemmer = SnowballStemmer("english")

In [ ]:
# Use WordNetLemmatizer instead of stemmer
#from nltk.stem import WordNetLemmatizer
#lemmatizer = WordNetLemmatizer()
#df['Lemmatized'] = df['StopRemoved'].apply(lambda x: [lemmatizer.lemmatize(y) for y in x])

다음으로 NLTK의 일부분인 [Snowball Stemmer](http://snowball.tartarus.org/)를 임포트한다. [Stemming](http://en.wikipedia.org/wiki/Stemming)은 그 근원으로 단어를 부수는 과정일 뿐이다.<br>

In [ ]:
# load nltk's SnowballStemmer as variabled 'stemmer'
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

아래에는 두 개의 함수를 정의했다:

<ul>
<li> *tokenize_and_stem*: tokenizes (시놉시스를 split해서 각 단어별(또는 토큰별) 리스트로 만들고, 각 토큰별 stems도 그렇게 한다.)
<li> *tokenize_only*: 시놉시스만 tokenizes 한다.
</ul>

사전을 만들기 위해 양쪽 함수를 쓰는데, 알고리즘을 위해 stems를 사용하고 싶을때 중요해지게 될거야. 하지만 나중에는 presentation을 위해 stems를 다시 전체 단어로 돌릴거다.

In [ ]:
# ORIGINAL CODE
# here I define a tokenizer and stemmer which returns the set of stems in the text that it is passed

def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems


def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

In [ ]:
# Define a tokenizer and stemmer which returns the set of stems in the text that it is passed
def tokenize_and_stem(text):
    # Remove punctuation
#    text = text.translate(str.maketrans('','',string.punctuation))

    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
#    tokens = [word for sent in nltk.sent_tokenize(text) for word in tokenizer.tokenize(sent.lower().split())]
    
    # MWETokenizer: manually link words, when disabled, use n-gram range in TF-IDF
#    tokens = tokenizer.tokenize(text.lower().split())
    tokens = tokenizer.tokenize(tokens)

    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if (re.search('[a-zA-Z]', token)): 
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    # WordNetLemmatizer
#    stems = [lemmatizer.lemmatize(t) for t in filtered_tokens]
    return stems


def tokenize_only(text):
    # Remove punctuation
#    text = text.translate(str.maketrans('','',string.punctuation))

    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
#    tokens = [word for sent in nltk.sent_tokenize(text) for word in tokenizer.tokenize(sent.lower().split())]
    
    # MWETokenizer: manually link words, when disabled, use n-gram range in TF-IDF
#    tokens = tokenizer.tokenize(text.lower().split())
    tokens = tokenizer.tokenize(tokens)

    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if (re.search('[a-zA-Z]', token)): 
            filtered_tokens.append(token)
    stems = filtered_tokens
#    stems = [stemmer.stem(t) for t in filtered_tokens]
    # WordNetLemmatizer
#    stems = [lemmatizer.lemmatize(t) for t in filtered_tokens]
    return stems


아래는 두 vocabularies 생성을 위해 시놉시스의 리스트들을 반복하는 tokenizing 함수들과 stemming/tokenizing을 사용한 것이다 : 하나는 stemmed 그리고 하나는 tokenized만을 위해

In [ ]:
totalvocab_stemmed = []
totalvocab_tokenized = []
for i in texts:
    allwords_stemmed = tokenize_and_stem(i)
    totalvocab_stemmed.extend(allwords_stemmed)
    
    allwords_tokenized = tokenize_only(i)
    totalvocab_tokenized.extend(allwords_tokenized)

이 두개의 리스트를 사용한다. 인덱스로 사용하기 위해 stemmed vocabulary로 판다스 DataFrame을 생성한다. tokenized 단어들은 칼럼으로 한다. 이 방식의 장점은 stem을 찾고 전체 토큰을 반환하는 효율적인 길을 제공한다는 점이다. 여기 아랫쪽은 stems를 토큰으로 한다(one to many) : stem 'run'은 'ran','runs','running' 등등과 관련이 있을 수 있다. 내 목적에 부합한다 -- 내가 찾아야 했던 stem과 관련있는 첫번째 토큰이 반환되어서 완전 행복하다<br>

In [ ]:
vocab_frame = pd.DataFrame({'words': totalvocab_tokenized}, index = totalvocab_stemmed)

In [ ]:
vocab_frame = pd.DataFrame({'words': totalvocab_tokenized}, index = totalvocab_stemmed)
print('there are ' + str(vocab_frame.shape[0]) + ' items in vocab_frame')

확실하게 몇몇 증복된 것이 있음을 알 수 있을 것이다. 정리할거당. 하지만 DataFrame에는 312209개의 아이템만이 있고, stem-index 기반의 stemmed word를 찾는 건 그리 어려운 일이 아니다.

In [ ]:
vocab_frame.head(29)

## Tf-idf와 문서 유사도

<img src='http://www.jiem.org/index.php/jiem/article/viewFile/293/252/2402' align='right' style="margin-left:10px">

여기 tf-idf, 텀 빈도수(term frequency)-역문서빈도(inverse document frequency) 벡터라이저 매개변수(vectorizer parameters)들을 정의해두었다. 그리고 *시놉시스*들 리스트를 tf-idf 행렬로 전환한다.<br>

Tf-idf 행렬을 얻기 위해, 첫번째로 문서에서 단어가 몇번 나왔는지를 센다. 이걸 document-term matrix(dtm)로 변환한다. dtm은 term frequency matrix라고도 부른다. dtm의 예시가 여기 있다.<br>

그리고 tf-idf(term frequency-inverse document frequency) weighting을 적용한다 : 문서안에서 빈번하게 등장하지만, corpus 안에서는 빈번하지 않은 단어들은 문서와 연관된 의미를 더 갖고있다 여겨지기에 더 높은 가중치를 받는다.<br>

내가 아래에 정의한 파라미터들에 대한 몇가지 노트:<br>


<ul>
<li>max_df: 이건 given feature가 가진 문서안에 있는 maximum frequency인데, tf-idf matrix안에서 사용될 수 있다. 만약 term이 문서에서 80% 이상이면 별 의미가 없는거라 볼 수 있다. (영화 시놉시스의 맥락에서)

<li> min_idf: 이건 정수일 수 있고 문서에서 term이 최소 5번 이상인 것들만 고려한다. 여기 0.2는; term은 반드시 문서상에서 최소 20%는 되어야 한다. 이름 기반 클러스터링을 한 끝에 발견했다. 만약 이보다 더 낮은 min_df을 허용한다면? -- 예로 "Michael" 또는 "Tom"등의 이름은 영화와 시놉시스등에 흔히 사용되지만, 이 이름들엔 어떤 유의미한 것도 담겨있지 않다.
<li> ngram_range: 이건 단지 unigrams(1개),bigrams(2개)와 trigrams(3개)를 보겠다는 의미다. 참고 [n-grams](http://en.wikipedia.org/wiki/N-gram)
</ul>

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=200000,
                                 min_df=0.2, stop_words='english',
                                 use_idf=True, tokenizer=tokenize_and_stem, ngram_range=(1,3))

%time tfidf_matrix = tfidf_vectorizer.fit_transform(texts)

print(tfidf_matrix.shape)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_df=0.9, max_features=200000,
                                   min_df=0.1, stop_words=my_stop_words, 
                                   use_idf=True, tokenizer=tokenize_and_stem, ngram_range=(1,3))

%time tfidf_matrix = tfidf_vectorizer.fit_transform(texts)

print(tfidf_matrix.shape)


*terms*는 tf-idf matrix안에서 사용된 특징(features)들의 리스트이다. 이건 vocabulary이다.

In [ ]:
terms = tfidf_vectorizer.get_feature_names()

In [ ]:
# terms

*dist*는 (1 - 각 문서의 cosine similarity)로 정의된다. Cosine similarity는 tf-idf matrix에 반해 측정되며, corpus안의 다른 문서들과 각 문서 사이의 유사도를 측정해서 생성하는데에 사용할 수 있다(시놉시스 사이의 각 시놉시스). 1에서 빼는 건 2차원 euclidean plane에 plotting하는데 쓸 수 있는 cosine distance를 제공한다.<br>

*dist*는 어떤 둘, 또는 그 이상가는 시놉시스간의 유사도라도 측정 가능하다는 걸 기억해두자.<br>

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
dist = 1 - cosine_similarity(tfidf_matrix)

# K-means 클러스터링

이제 재미있는 부분이다. tf-idf matrix를 사용해 시놉시스 내부 hidden structure를 더 잘 이해하기 위해, 다수의 클러스터링 알고리즘을 실행시킬 수 있다. 나는 첫번째로 [k-means](http://en.wikipedia.org/wiki/K-means_clustering)를 선택했다. K-means는 사전 결정된 클러스터들의 개수를 initialize한다(나는 5를 선택했다). 각 관측은 클러스터에 적용되고(cluster assignment) 클러스터 sum of squares에 minimize된다. 다음으로, 클러스터된 관측(clustered observation)들의 평균(mean)은 측정되어서 새로운 클러스터 중심점(cluster centroid)에 사용된다. 그리고 반복되는 과정에서 관측된 건 다시 클러스터들에 적용되고 centroids도 다시 계산된다. 알고리즘이 수렴할때까지.<br>

global optimum에 수렴하도록 k-means로 알고리즘을 돌려보다, k-means가 local optima에 영향을 받기 쉽다는 걸 발견했다.

In [ ]:
from sklearn.cluster import KMeans
num_clusters = 5
km = KMeans(n_clusters=num_clusters)
%time km.fit(tfidf_matrix)
clusters = km.labels_.tolist()

model을 pickle하려고 joblib.dump를 사용했다, 한번 수렴했고 model/reassign을의 labels를 cluster로서 reload 한다. 

In [ ]:
from sklearn.externals import joblib

# joblib.dump(km,  'doc_cluster.pkl')
km = joblib.load('doc_cluster.pkl')
clusters = km.labels_.tolist()

In [ ]:
df_tfidf_matrix = pd.DataFrame(tfidf_matrix.toarray())
# df_tfidf_matrix.head()

여기 titles, ranks, synopsis, cluster assignment, genre 사전을 만든다 [rank와 genre는 IMBDB에서 스크랩했다]<br>

쉽게 접근하기 위해 사전을 판다스 DataFrame으로 변환한다. 난 [Pandas](http://pandas.pydata.org/)의 엄청난 팬이고 내가 아래에 해놓은 것처럼 끝내주는 funtionality들을 보기를 추천한다. 하지만 자세한 것은 설명하지 않겠다.<br>

In [ ]:
news = {'date': dates,'articlecode': articlecodes, 'title': titles,'text': texts,'cluster': clusters}
frame = pd.DataFrame(news, index = [clusters], columns = ['date','articlecode','title','text','cluster'])

In [ ]:
frame['cluster'].value_counts() #number of films per cluster (clusters from 0 to 4)

# Export to Access to analyze clusters
# frame.to_csv('newsclusters.txt', sep='^')

cluster centroid 근처의 상위 *n* (나는 *n*=6으로 했다) 단어들을 식별하기 위해 여기 각 클러스터별 몇 가지 간단한 색인과 분류를 하자. 이건 클러스터의 메인 토픽에 대한 good sense를 제공한다.

In [ ]:
from __future__ import print_function

print("Top terms per cluster:")
print()
order_centroids = km.cluster_centers_.argsort()[:, ::-1]
for i in range(num_clusters):
    print("Cluster %d words:" % i, end='')
    for ind in order_centroids[i, :6]:
        print(' %s' % vocab_frame.ix[terms[ind].split(' ')].values.tolist()[0][0].encode('utf-8', 'ignore'), end=',')
    print()
    print()
#     print("Cluster %d titles:" % i, end='')
#     for title in frame.ix[i]['title'].values.tolist():
#         print(' %s,' % title, end='')
#     print()
#     print()

In [ ]:
#export tables to HTML
print(frame[['Rank', 'Title']].loc[frame['cluster'] == 1].to_html(index=False))

# 다차원 스케일링 (Multidimensional scaling)

여기 [multidimensional scaling](http://en.wikipedia.org/wiki/Multidimensional_scaling)를 사용해서 dist matrix를 2차원 array로 변환하기 위한 코드가 있다. MDS에 대해 많이 아는 척을 하려는 건 아니지만, 이 목적에 유용하다. 다른 옵션으로 [principal component analysis](http://en.wikipedia.org/wiki/Principal_component_analysis)를 쓸 수 있다.

In [ ]:
import os  # for os.path.basename

import matplotlib.pyplot as plt
import matplotlib as mpl

from sklearn.manifold import MDS

MDS()

# two components as we're plotting points in a two-dimensional plane
# "precomputed" because we provide a distance matrix
# we will also specify `random_state` so the plot is reproducible.
mds = MDS(n_components=2, dissimilarity="precomputed", random_state=1)

pos = mds.fit_transform(dist)  # shape (n_components, n_samples)

xs, ys = pos[:, 0], pos[:, 1]

In [ ]:
#strip any proper nouns (NNP) or plural proper nouns (NNPS) from a text
from nltk.tag import pos_tag

def strip_proppers_POS(text):
    tagged = pos_tag(text.split()) #use NLTK's part of speech tagger
    non_propernouns = [word for word,pos in tagged if pos != 'NNP' and pos != 'NNPS']
    return non_propernouns

## 문서 클러스터 보이기 (Visualizing document clusters)

이 섹션에서는 matplotlib과 mpld3를 사용해서 어떻게 문서 클러스터링 결과를 보일 수 있는지 시연한다. (a matplotlib wrapper for D3.js). 클러스터 숫자로부터 색깔과 클러스터 이름으로 가기 위해, 첫번째로 몇개의 사전을 정의한다.

기본적으로 cluster names, 각 cluster centroid에 가장 가까운 단어들을 껐다.

In [ ]:
#set up colors per clusters using a dict
cluster_colors = {0: '#1b9e77', 1: '#d95f02', 2: '#7570b3', 3: '#e7298a', 4: '#66a61e'}

#set up cluster names using a dict
cluster_names = {0: 'Family, home, war', 
                 1: 'Police, killed, murders', 
                 2: 'Father, New York, brothers', 
                 3: 'Dance, singing, love', 
                 4: 'Killed, soldiers, captain'}

다음으로 matplotlib을 사용해 클러스터에서 labeled observations(영화, 영화 타이틀) 색깔넣기를 한다. matplolib plot에 대해 너무 상세하게는 안 할거지만, 몇몇 도움되는 것들을 알려주겠다.

In [ ]:
%matplotlib inline

In [ ]:
#create data frame that has the result of the MDS plus the cluster numbers and titles
df = pd.DataFrame(dict(x=xs, y=ys, label=clusters, title=titles)) 

#group by cluster
groups = df.groupby('label')


# set up plot
fig, ax = plt.subplots(figsize=(17, 9)) # set size
ax.margins(0.05) # Optional, just adds 5% padding to the autoscaling

#iterate through groups to layer the plot
#note that I use the cluster_name and cluster_color dicts with the 'name' lookup to return the appropriate color/label
for name, group in groups:
    ax.plot(group.x, group.y, marker='o', linestyle='', ms=12, label=cluster_names[name], color=cluster_colors[name], mec='none')
    ax.set_aspect('auto')
    ax.tick_params(\
        axis= 'x',          # changes apply to the x-axis
        which='both',      # both major and minor ticks are affected
        bottom='off',      # ticks along the bottom edge are off
        top='off',         # ticks along the top edge are off
        labelbottom='off')
    ax.tick_params(\
        axis= 'y',         # changes apply to the y-axis
        which='both',      # both major and minor ticks are affected
        left='off',      # ticks along the bottom edge are off
        top='off',         # ticks along the top edge are off
        labelleft='off')
    
ax.legend(numpoints=1)  #show legend with only 1 point

#add label in x,y position with the label as the film title
for i in range(len(df)):
    ax.text(df.ix[i]['x'], df.ix[i]['y'], df.ix[i]['title'], size=8)  

    
    
plt.show() #show the plot

#uncomment the below to save the plot if need be
#plt.savefig('clusters_small_noaxes.png', dpi=200)

In [ ]:
plt.close()

clustering plot은 끝내주는 것 처럼 보이지만, overlapping labels를 보고 있자니 눈이 아프다. [D3.js](http://d3js.org/)를 써보자. 한가지 방책은 based/javascript interactive 브라우저를 사용하는 것임을 알고 있다. 다행히 최근에 D3를 위한 matplotlib wrapper인 [mpld3](https://mpld3.github.io/)를 발견했다. Mpld3는 기본적으로 web interactives 생성을 위해 matplotlib 문법을 사용할 수 있게 해준다. 마우스 돌리기에 툴팁을 추가하기 위한 고수준 API는 정말 쉽다.<br>

또한 zooming과 panning에 잘 동작하기도 한다. 아래 자바스크립트 코드 조각은 기본적으로 어디를 zoom/pan toggle할지 custom location을 정의한다. 너무 걱정마라. 그걸 쓸 필요는 없으니까. 하지만 나중에 web을 exporting 할때 formatting purposes를 도와준다. 넌 아마 toolbar 위치를 위한 x와 y attribute를 바꾸길 바랄거다.

In [ ]:
#define custom toolbar location
class TopToolbar(mpld3.plugins.PluginBase):
    """Plugin for moving toolbar to top of figure"""

    JAVASCRIPT = """
    mpld3.register_plugin("toptoolbar", TopToolbar);
    TopToolbar.prototype = Object.create(mpld3.Plugin.prototype);
    TopToolbar.prototype.constructor = TopToolbar;
    function TopToolbar(fig, props){
        mpld3.Plugin.call(this, fig, props);
    };

    TopToolbar.prototype.draw = function(){
      // the toolbar svg doesn't exist
      // yet, so first draw it
      this.fig.toolbar.draw();

      // then change the y position to be
      // at the top of the figure
      this.fig.toolbar.toolbar.attr("x", 150);
      this.fig.toolbar.toolbar.attr("y", 400);

      // then remove the draw function,
      // so that it is not called again
      this.fig.toolbar.draw = function() {}
    }
    """
    def __init__(self):
        self.dict_ = {"type": "toptoolbar"}

여기 interactive scatterplot의 actual creation이 있다. cluster로 group하기 위해 pandas의 groupgby를 씀에도, 이는 mpld3 예시 하나의 직접적인 copy이다. 그리고 그룹들을 통과하며 scatterplot 층처럼 반복한다. raw D3, mpld3와 이걸 하는 건, python workflow로 접목하는 것보다 더 심플한 것과 관련있다.<br>

만약 니가 내 웹사이트 전반을 둘려보려 한다면 내가 D3를 사랑한다는 걸 알게될것이다, 하지만 기본 interactives를 위해 나는 많이 나가기 위해 mpld3를 사용할 수도 있다. mpld3는 내가 형태에 폰트 스타일, 축들, 왼쪽 여백을 사용햇듯, 니가 일부 커스텀 CSS를 define하게 해준다.

In [ ]:
#create data frame that has the result of the MDS plus the cluster numbers and titles
df = pd.DataFrame(dict(x=xs, y=ys, label=clusters, title=titles)) 

#group by cluster
groups = df.groupby('label')

#define custom css to format the font and to remove the axis labeling
css = """
text.mpld3-text, div.mpld3-tooltip {
  font-family:Arial, Helvetica, sans-serif;
}

g.mpld3-xaxis, g.mpld3-yaxis {
display: none; }
"""

# Plot 
fig, ax = plt.subplots(figsize=(14,6)) #set plot size
ax.margins(0.03) # Optional, just adds 5% padding to the autoscaling

#iterate through groups to layer the plot
#note that I use the cluster_name and cluster_color dicts with the 'name' lookup to return the appropriate color/label
for name, group in groups:
    points = ax.plot(group.x, group.y, marker='o', linestyle='', ms=18, label=cluster_names[name], mec='none', color=cluster_colors[name])
    ax.set_aspect('auto')
    labels = [i for i in group.title]
    
    #set tooltip using points, labels and the already defined 'css'
    tooltip = mpld3.plugins.PointHTMLTooltip(points[0], labels,
                                       voffset=10, hoffset=10, css=css)
    #connect tooltip to fig
    mpld3.plugins.connect(fig, tooltip, TopToolbar())    
    
    #set tick marks as blank
    ax.axes.get_xaxis().set_ticks([])
    ax.axes.get_yaxis().set_ticks([])
    
    #set axis as blank
    ax.axes.get_xaxis().set_visible(False)
    ax.axes.get_yaxis().set_visible(False)

    
ax.legend(numpoints=1) #show legend with only one dot

mpld3.display() #show the plot

#uncomment the below to export to html
#html = mpld3.fig_to_html(fig)
#print(html)

# 계층적 문서 클러스터링 (Hierarchical document clustering)

이제 k-means를 사용해서 cluster와 문서 plot하는 일을 성공적으로 해냈다, 다른 클러스터링 알고리즘을 시험해보자. 난 [Ward clustering algorithm](http://en.wikipedia.org/wiki/Ward%27s_method)을 선택했다. 계층적 클러스터링을 제공하기 때문이다. Ward clustering은 집적적인 클러스터링 방법이다. 각 스테이지별로 최소의 between-cluster 거리를 가진 클러스터 쌍(pair of clusters)을 합쳐준다. 미리 계산된 cosine distance matrix (*dist*)를 linkage_matrix를 계산하는데에 사용하고, dendrogram을 plot할거다.<br>

기억할 것, 이 방법은 3개의 주요 클러스터를 반환한다. 가장 큰 클러스터는 4개의 주요 서브 클러스터로 쪼갠다. 붉은색 클러스터는 "killed, soldiers, captain" 영화의 많은 부분을 담고 있다.

In [ ]:
from scipy.cluster.hierarchy import ward, dendrogram

linkage_matrix = ward(dist) #define the linkage_matrix using ward clustering pre-computed distances

fig, ax = plt.subplots(figsize=(15, 20)) # set size
ax = dendrogram(linkage_matrix, orientation="right", labels=titles);

plt.tick_params(\
    axis= 'x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom='off',      # ticks along the bottom edge are off
    top='off',         # ticks along the top edge are off
    labelbottom='off')

plt.tight_layout() #show plot with tight layout

#uncomment below to save figure
plt.savefig('ward_clusters.png', dpi=200) #save figure as ward_clusters

In [ ]:
plt.close()

# 잠재 디리클레 할당 (Latent Dirichlet Allocation, LDA)

이 섹션은 상위 100 영화 시놉시스안의 hidden structure에 대해 더 배우기 위해 [Latent Dirichlet Allocation (LDA)](http://en.wikipedia.org/wiki/Latent_Dirichlet_allocation)를 사용하는데에 집중할거다. 

LDA는 probabilistic topic model인데, 문서 내부의 각 단어는 문서의 topic들에 attributable하고, 문서를 topic들이 섞인 것이라 가정한다. 이 필드에서 유명한 이름 중 하나, David Blei에 의해 된, probabilistic topic model들의 썩 괜찮은 high-level overview 이다. 여기서 [Communications of the ACM here](http://delivery.acm.org/10.1145/2140000/2133826/p77-blei.pdf?ip=68.48.185.120&id=2133826&acc=OPEN&key=4D4702B0C3E38B35%2E4D4702B0C3E38B35%2E4D4702B0C3E38B35%2E6D218144511F3437&CFID=612398453&CFTOKEN=48760790&__acm__=1419436704_2d47aefe0700e44f81eb822df659a341) 볼 수 있다. 덧붙이자면, Blie는 LDA의 seminal paper의 저자 중 하나이기도 하다. 

LDA의 적용을 위해 나는 [Gensim pacakage](https://radimrehurek.com/gensim/)를 사용한다. 시놉시스 전처리를 좀 다르게 할 것인데, 우선 어떤 고유 명사(proper noun)도 제거할 수 있는 함수를 정의했다.

In [ ]:
#strip any proper names from a text...unfortunately right now this is yanking the first word from a sentence too.
import string
def strip_proppers(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent) if word.islower()]
    return "".join([" "+i if not i.startswith("'") and i not in string.punctuation else i for i in tokens]).strip()

위의 함수는 오로지 capitalization에 기반한 것이라, 문장들 초반에 있는 단어 제거에 취약하다. 그래서, 나는 NLTK speech tagger의 일부를 사용해 아래 함수를 썼다. 하지만 전체 시놉시스를 돌리는데에 너무 오래 걸려서, 위에 갇혀버렷다. ;(

In [ ]:
#strip any proper nouns (NNP) or plural proper nouns (NNPS) from a text
from nltk.tag import pos_tag

def strip_proppers_POS(text):
    tagged = pos_tag(text.split()) #use NLTK's part of speech tagger
    non_propernouns = [word for word,pos in tagged if pos != 'NNP' and pos != 'NNPS']
    return non_propernouns

이건 내가 실제로 돌린 텍스트 프로세싱이다. (고유 명사 제거, tokenization, 불용어 제거)

In [ ]:
#Latent Dirichlet Allocation implementation with Gensim

from gensim import corpora, models, similarities 

#remove proper names
preprocess = [strip_proppers(doc) for doc in texts]

%time tokenized_text = [tokenize_and_stem(text) for text in preprocess]

%time texts = [[word for word in text if word not in my_stop_words] for text in tokenized_text]

In [ ]:
#print(len([word for word in texts[0] if word not in stopwords]))
print(len(texts[0]))

아래 세부적인 Gensim conversions들이 있다; 극단적인 단어들도 필터링했다 (코드 내부의 코멘트를 봐라)

In [ ]:
dictionary = corpora.Dictionary(texts)

In [ ]:
dictionary.filter_extremes(no_below=1, no_above=0.8)

In [ ]:
corpus = [dictionary.doc2bow(text) for text in texts]

In [ ]:
len(corpus)

실제 모델은 아래서 돌아간다. 확실한 수렴을 위해 100번의 pass가 걸렸다, 하지만 너는 내 컴퓨터가 돌아가는데 13분 걸린 걸 볼 수 있다. 내 chunksize는 corpus보다 크므로, 기본적으로 pass마다 모든 시놉시스를 사용한다. optimize해야 하는데, Gensim은 병렬로 돌릴 수 있는 capacity를 갖고 있다. 더 나아가 이걸 더 큰 corpora에 적용하는 모험을 할거다.

In [ ]:
%time lda = models.LdaModel(corpus, num_topics=5, id2word=dictionary, update_every=5, chunksize=10000, passes=100)

In [ ]:
%time lda = models.LdaModel(corpus, num_topics=5, 
                            id2word=dictionary, 
                            update_every=5, 
                            chunksize=10000, 
                            passes=100)

각 topic은 그걸 정의한 단어들의 set을 갖고 있으며, 명확한 확률을 따른다.

In [ ]:
lda.show_topics()

In [ ]:
print(lda[corpus[0]])

In [ ]:
topics = lda.print_topics(5, num_words=20)

여기, 나는 topics를 단지 각 topic별 상위 20 단어들의 리스트로 convert한다. 넌 war/family topic과 더 명확한 war/epic topic을 포함한 k-means를 사용해서 식별한 것과 비슷한 topics의 breakdown을 볼 수 있다.

In [ ]:
topics_matrix = lda.show_topics(formatted=False, num_words=20)

In [ ]:
topics_matrix = np.array(topics_matrix)

In [ ]:
topics_matrix.shape

In [ ]:
topic_words = topics_matrix[:,:,1]

In [ ]:
for i in topic_words:
    print([str(word) for word in i])
    print()